# Module 1 - Programming Assignment

## General Directions

1. You must follow the Programming Requirements outlined on Canvas.
2. The Notebook should be cleanly and fully executed before submission.
3. You should change the name of this file to be your JHED id. For example, `jsmith299.ipynb` although Canvas will change it to something else...
4. You must follow the Programming Requirments for this course.

<div style="background: lemonchiffon; margin:20px; padding: 20px;">
    <strong>Important</strong>
    <p>
        You should always read the entire assignment before beginning your work, so that you know in advance what the requested output will be and can plan your implementation accordingly.
    </p>
</div>

<div style="color: white; background: #C83F49; margin:20px; padding: 20px;">
    <strong>Academic Integrity and Copyright</strong>
    <p>You are not permitted to consult outside sources (Stackoverflow, YouTube, ChatGPT, etc.) or use "code assistance" (Co-Pilot, etc) to complete this assignment. By submitting this assignment for grading, you certify that the submission is 100% your own work, based on course materials, group interactions, instructor guidance. You agree to comply by the requirements set forth in the Syllabus, including, by reference, the JHU KSAS/WSE Graduate Academic Misconduct Policy.</p>
    <p>Sharing this assignment either directly (e.g., email, github, homework site) or indirectly (e.g., ChatGPT, machine learning platform) is a violation of the copyright. Additionally, all such sharing is a violation the Graduate Academic Misconduct Policy (facilitating academic dishonesty is itself academic dishonesty), even after you graduate.</p>
    <p>If you have questions or if you're unsure about the policy, ask via Canvas Inbox. In this case, being forgiven is <strong>not</strong> easier than getting permission and ignorance is not an excuse.</p>
    <p>This assignment is copyright (&copy Johns Hopkins University &amp; Stephyn G. W. Butcher). All rights reserved.</p>
</div>

# State Space Search with A* Search

You are going to implement the A\* Search algorithm for navigation problems.


## Motivation

Search is often used for path-finding in video games. Although the characters in a video game often move in continuous spaces,
it is trivial to layout a "waypoint" system as a kind of navigation grid over the continuous space. Then if the character needs
to get from Point A to Point B, it does a line of sight (LOS) scan to find the nearest waypoint (let's call it Waypoint A) and
finds the nearest, LOS waypoint to Point B (let's call it Waypoint B). The agent then does a A* search for Waypoint B from Waypoint A to find the shortest path. The entire path is thus Point A to Waypoint A to Waypoint B to Point B.

We're going to simplify the problem by working in a grid world. The symbols that form the grid have a special meaning as they
specify the type of the terrain and the cost to enter a grid cell with that type of terrain:

```
token   terrain    cost 
🌾       plains     1
🌲       forest     3
⛰       hills      5
🐊       swamp      7
🌋       mountains  impassible
```

We can think of the raw format of the map as being something like:

```
🌾🌾🌾🌾🌲🌾🌾
🌾🌾🌾🌲🌲🌲🌾
🌾⛰⛰⛰🌾🌾🌾
🌾🌾⛰⛰🌾🌾🌾
🌾🌾⛰🌾🌾🌲🌲
🌾🌾🌾🌾🌲🌲🌲
🌾🌾🌾🌾🌾🌾🌾
```

## The World

Given a map like the one above, we can easily represent each row as a `List` and the entire map as `List of Lists`:

In [1]:
full_world = [
    ['🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🐊', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '⛰', '⛰', '⛰'], 
    ['🌾', '🌾', '🌾', '🌾', '🌾', '⛰', '⛰', '🌾', '🌾', '🌾', '🌾', '🐊', '🐊', '🐊', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '⛰', '🌋', '🌋', '⛰'], 
    ['🌾', '🌾', '🌾', '🌾', '🌾', '⛰', '⛰', '⛰', '🌾', '🌾', '🐊', '🐊', '🐊', '🐊', '🌾', '🌾', '🌋', '🌾', '🌾', '🌾', '⛰', '🌾', '🌾', '⛰', '⛰', '🌋', '🌾'], 
    ['🌾', '🌾', '🌾', '🌾', '⛰', '⛰', '🌋', '⛰', '⛰', '🐊', '🐊', '🐊', '🐊', '🐊', '🌾', '🌋', '🌋', '🌋', '🌋', '🌾', '⛰', '🌾', '🌾', '🌾', '⛰', '🌋', '🌾'], 
    ['🌾', '🌾', '🌋', '⛰', '⛰', '🌋', '🌋', '⛰', '⛰', '🐊', '🐊', '🐊', '🐊', '🐊', '🌋', '🌋', '🌲', '🌋', '🌋', '🌋', '⛰', '⛰', '🌾', '🌾', '⛰', '⛰', '🌾'], 
    ['🌲', '🌾', '🌋', '🌋', '🌋', '🌋', '⛰', '⛰', '⛰', '🐊', '🐊', '🐊', '🌾', '🌾', '🌾', '🌋', '🌲', '🌲', '🌋', '🌋', '⛰', '⛰', '🌾', '⛰', '⛰', '🌾', '🌾'], 
    ['🌲', '🌾', '🌲', '🌋', '🌋', '⛰', '⛰', '⛰', '🌾', '🌾', '🐊', '🌾', '🌾', '🌾', '🌾', '🌲', '🌲', '🌲', '🌲', '🌋', '🌋', '⛰', '⛰', '⛰', '🌾', '🌾', '🌾'], 
    ['🌲', '🌲', '🌲', '🌋', '🌲', '⛰', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '⛰', '⛰', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌋', '🌋', '⛰', '⛰', '🌾', '🌾', '🌾'], 
    ['🌲', '🌲', '🌲', '🌲', '🌲', '🌾', '🌾', '🌾', '🌾', '⛰', '⛰', '⛰', '⛰', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌋', '⛰', '⛰', '🌾', '🌾', '🌾'], 
    ['🌲', '🌲', '🌲', '🌲', '🌾', '🌾', '🌾', '🌾', '🌾', '⛰', '⛰', '🌋', '🌋', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌋', '🌋', '⛰', '🌾', '🌾', '🌾'], 
    ['🌲', '🌲', '🌲', '🌲', '🌾', '🌾', '🌾', '🌾', '🌾', '⛰', '🌋', '🌋', '🌋', '⛰', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌋', '🌾', '🌾', '⛰', '🌾'], 
    ['🌲', '🌲', '🌲', '🌲', '🐊', '🌾', '⛰', '🌾', '🌾', '🌋', '🌋', '⛰', '⛰', '🌾', '⛰', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌋', '🌾', '🌋', '⛰', '🌾'], 
    ['🌲', '🌲', '🌲', '🐊', '🐊', '🐊', '🌋', '🌾', '⛰', '🌋', '🌋', '🌾', '🌾', '🌾', '⛰', '🌋', '🌲', '🌲', '🌲', '🌲', '🌲', '🌋', '🌋', '🌾', '🌋', '🌋', '⛰'], 
    ['🌲', '🌲', '🌲', '🐊', '🐊', '🐊', '🌋', '⛰', '⛰', '🌋', '⛰', '🌾', '🐊', '🌾', '⛰', '🌋', '🌲', '🌲', '🌲', '🌾', '🌾', '🌋', '🌾', '🌾', '🌋', '🌋', '⛰'], 
    ['🌲', '🌲', '🌲', '🌲', '🐊', '🐊', '🌋', '🌋', '🌋', '🌋', '🌾', '🌾', '🐊', '🌾', '⛰', '🌋', '🌋', '🌲', '🌾', '🌾', '🌋', '🌾', '🌾', '🌾', '⛰', '🌋', '⛰'], 
    ['🌾', '🌲', '🌲', '🌲', '🌲', '🐊', '🐊', '🌋', '🌋', '🌾', '🌾', '🌾', '🐊', '🐊', '🌾', '⛰', '🌋', '🌲', '🌾', '🌾', '🌾', '🌾', '🌾', '⛰', '⛰', '🌋', '⛰'], 
    ['🌾', '🌾', '🌲', '🌲', '🌲', '🐊', '🐊', '🌋', '🌾', '🌾', '🌾', '🐊', '🐊', '🐊', '🐊', '⛰', '🌋', '🌋', '🌾', '🌾', '🌾', '🌾', '🌾', '⛰', '🌋', '⛰', '⛰'], 
    ['🌾', '🌾', '🌋', '🌲', '🌲', '🌾', '🐊', '🐊', '🐊', '🌾', '🌾', '🐊', '🌾', '🐊', '🐊', '🌾', '🌾', '🌋', '⛰', '🌾', '🌾', '🌾', '⛰', '🌋', '🌋', '⛰', '🌾'], 
    ['🌾', '🌋', '🌋', '🌲', '🌾', '🌾', '🌾', '🐊', '🐊', '🐊', '🌾', '🌾', '🌾', '🐊', '🐊', '🐊', '🌾', '🌋', '⛰', '🌾', '🌾', '🌾', '⛰', '🌋', '🌾', '⛰', '🌾'], 
    ['🌾', '🌋', '🌋', '🌾', '🌾', '🌾', '🌾', '🐊', '🌾', '🌾', '⛰', '🌾', '🌾', '🌾', '🌾', '🌾', '🌋', '🌋', '🌾', '🌾', '🌾', '🌾', '🌾', '⛰', '🌋', '⛰', '🌾'], 
    ['🌾', '🌋', '⛰', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '⛰', '🌋', '⛰', '⛰', '🌾', '🌾', '⛰', '🌋', '🌾', '🌾', '🌾', '🐊', '🐊', '🌾', '⛰', '🌋', '🌋', '🌾'], 
    ['🌾', '🌋', '⛰', '⛰', '⛰', '🌾', '🌾', '🌾', '⛰', '⛰', '🌋', '🌋', '🌋', '⛰', '⛰', '🌋', '🌋', '🌾', '🌾', '🌾', '🐊', '🐊', '🐊', '🌾', '⛰', '🌋', '⛰'], 
    ['🌾', '🌋', '⛰', '⛰', '🌋', '⛰', '🌾', '⛰', '⛰', '⛰', '🌋', '🌋', '⛰', '🌾', '🌋', '🌋', '🌾', '🌾', '🌾', '🌾', '🐊', '🐊', '🐊', '🐊', '⛰', '🌋', '⛰'], 
    ['🌾', '🌋', '🌋', '🌋', '🌋', '🌋', '⛰', '⛰', '⛰', '🌾', '⛰', '⛰', '🌾', '🌾', '⛰', '⛰', '🌾', '🌾', '🌾', '🐊', '🐊', '🐊', '🐊', '🐊', '🐊', '🐊', '⛰'], 
    ['🌾', '🌋', '🌋', '🌋', '🌋', '⛰', '🌾', '⛰', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🐊', '🐊', '🐊', '🐊', '🐊', '🐊', '🐊', '🌾'], 
    ['🌾', '🌾', '⛰', '⛰', '⛰', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🐊', '🐊', '🐊', '🐊', '🐊', '🐊', '🐊', '🌾'], 
    ['🌾', '🌾', '⛰', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🐊', '🐊', '🐊', '🐊', '🐊', '🐊', '🐊', '🌾']
]

Note: emojis are not fixed width so here is a function that will print out a List of Lists of emojis in an HTML table.

In [2]:
from IPython.display import display_html

def display_emoji_grid(emoji_grid):
    """
    Display a List of Lists of emojis in a perfect grid (table) in a Jupyter Notebook.
    
    Parameters:
    emoji_grid (list of list of str): A 2D list containing emojis to display in a grid.
    """
    # Create HTML table
    html = '<table style="border-collapse: collapse;">'
    
    for row in emoji_grid:
        html += '<tr>'
        for emoji in row:
            html += f'<td style="border: none; padding: 0px; text-align: center; font-size: 1em;">{emoji}</td>'
        html += '</tr>'
    
    html += '</table>'
    
    # Display the HTML table
    display_html(html, raw=True)
#

In [3]:
display_emoji_grid(full_world)

🌾,🌾,🌾,🌾,🌾,🌾,🌾,🌾,🌾,🌾,🌾,🌾,🐊,🌾,🌾,🌾,🌾,🌾,🌾,🌾,🌾,🌾,🌾,🌾,⛰,⛰,⛰
🌾,🌾,🌾,🌾,🌾,⛰,⛰,🌾,🌾,🌾,🌾,🐊,🐊,🐊,🌾,🌾,🌾,🌾,🌾,🌾,🌾,🌾,🌾,⛰,🌋,🌋,⛰
🌾,🌾,🌾,🌾,🌾,⛰,⛰,⛰,🌾,🌾,🐊,🐊,🐊,🐊,🌾,🌾,🌋,🌾,🌾,🌾,⛰,🌾,🌾,⛰,⛰,🌋,🌾
🌾,🌾,🌾,🌾,⛰,⛰,🌋,⛰,⛰,🐊,🐊,🐊,🐊,🐊,🌾,🌋,🌋,🌋,🌋,🌾,⛰,🌾,🌾,🌾,⛰,🌋,🌾
🌾,🌾,🌋,⛰,⛰,🌋,🌋,⛰,⛰,🐊,🐊,🐊,🐊,🐊,🌋,🌋,🌲,🌋,🌋,🌋,⛰,⛰,🌾,🌾,⛰,⛰,🌾
🌲,🌾,🌋,🌋,🌋,🌋,⛰,⛰,⛰,🐊,🐊,🐊,🌾,🌾,🌾,🌋,🌲,🌲,🌋,🌋,⛰,⛰,🌾,⛰,⛰,🌾,🌾
🌲,🌾,🌲,🌋,🌋,⛰,⛰,⛰,🌾,🌾,🐊,🌾,🌾,🌾,🌾,🌲,🌲,🌲,🌲,🌋,🌋,⛰,⛰,⛰,🌾,🌾,🌾
🌲,🌲,🌲,🌋,🌲,⛰,🌾,🌾,🌾,🌾,🌾,🌾,⛰,⛰,🌲,🌲,🌲,🌲,🌲,🌲,🌋,🌋,⛰,⛰,🌾,🌾,🌾
🌲,🌲,🌲,🌲,🌲,🌾,🌾,🌾,🌾,⛰,⛰,⛰,⛰,🌲,🌲,🌲,🌲,🌲,🌲,🌲,🌲,🌋,⛰,⛰,🌾,🌾,🌾
🌲,🌲,🌲,🌲,🌾,🌾,🌾,🌾,🌾,⛰,⛰,🌋,🌋,🌲,🌲,🌲,🌲,🌲,🌲,🌲,🌲,🌋,🌋,⛰,🌾,🌾,🌾
🌲,🌲,🌲,🌲,🌾,🌾,🌾,🌾,🌾,⛰,🌋,🌋,🌋,⛰,🌲,🌲,🌲,🌲,🌲,🌲,🌲,🌲,🌋,🌾,🌾,⛰,🌾


## Warning

One implication of this representation is that (x, y) is world[ y][ x] so that (3, 2) is world[ 2][ 3] and world[ 7][ 9] is (9, 7). Yes, there are many ways to do this. I picked this representation because when you look at it, it *looks* like a regular x, y cartesian grid and it's easy to print out.

It is often easier to begin your programming by operating on test input that has an obvious solution. If we had a small 7x7 world with the following characteristics:

In [4]:
small_world = [
    ['🌾', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲'],
    ['🌾', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲'],
    ['🌾', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲'],
    ['🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾'],
    ['🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌾'],
    ['🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌾'],
    ['🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌾']
]

**what do you expect the policy would be?** Think about it for a bit. This will help you with your programming and debugging.

## States and State Representation

The canonical pieces of a State Space Search problem are the States, Actions, Transitions and Costs. 

We'll start with the state representation. For the navigation problem, a state is the current position of the agent, `(x,y)`. The entire set of possible states is implicitly represented by the world map.

## Actions and Transitions

Next we need to specify the actions. In general, there are a number of different possible action sets in such a world. The agent might be constrained to move north/south/east/west or diagonal moves might be permitted as well (or really anything). When combined with the set of States, the *permissible* actions forms the Transition set.

Rather than enumerate the Transition set directly, for this problem it's easier to calculate the available actions and transitions on the fly. This can be done by specifying a *movement model* as offsets to the current state and then checking to see which of the potential successor states are actually permitted. This can be done in the successor function mentioned in the pseudocode.

One such example of a movement model is shown below.

In [5]:
MOVES = [(0,-1), (1,0), (0,1), (-1,0)]

## Costs

We can encode the costs described above in a `Dict`:

In [6]:
COSTS = { '🌾': 1, '🌲': 3, '⛰': 5, '🐊': 7}

## Specification

You will implement a function called `a_star_search` that takes the parameters and returns the value as specified below. The return value is going to look like this:

`[(0,1), (0,1), (0,1), (1,0), (1,0), (1,0), (1,0), (1,0), (1,0), (0,1), (0,1), (0,1)]`

You should also implement a function called `pretty_print_path`. 
The `pretty_print_path` function prints an ASCII representation of the path generated by the `a_star_search` on top of the terrain map. 
For example, for the test world, it would print this:

```
⏬🌲🌲🌲🌲🌲🌲
⏬🌲🌲🌲🌲🌲🌲
⏬🌲🌲🌲🌲🌲🌲
⏩⏩⏩⏩⏩⏩⏬
🌲🌲🌲🌲🌲🌲⏬
🌲🌲🌲🌲🌲🌲⏬
🌲🌲🌲🌲🌲🌲🎁
```

using ⏩,⏪,⏫ ⏬ to represent actions and `🎁` to represent the goal. (Note the format of the output...there are no spaces, commas, or extraneous characters). You are printing the path over the terrain.
This is an impure function (because it has side effects, the printing, before returning anything).

* Do not print raw data structures
* Do not insert unneeded/unrequested spaces
* Use the provided emojis!
* Use the provided `display_emoji_grid` function.

### Additional comments

As Python is an interpreted language, you're going to need to insert all of your functions *before* the actual `a_star_search` function implementation. 
Do not make unwarranted assumptions (for example, do not assume that the start is always (0, 0).
Do not refer to global variables, pass them as parameters (functional programming).

Simple and correct is better than inefficient and incorrect, or worse, incomplete.
For example, you can use a simple List, with some helper functions, as a Stack or a Queue or a Priority Queue.
Avoid the Python implementations of HeapQ, PriorityQueue implementation unless you are very sure about what you're doing as they require *immutable* keys.

In [7]:
from typing import List, Tuple, Dict, Callable
from copy import deepcopy

*add as many markdown and code cells here as you need for helper functions. We have added `heuristic` for you*

<a id="get_possible_moves"></a>
## `get_possible_moves`

This function calculates all possible moves from the current position in a given world grid. The possible moves are determined by adding predefined move directions to the current position coordinates. Once these new coordinates are generated they are then checked to ensure they are not out of bounds of the map.

### Parameters:
- `current_position` (Tuple[int, int]): The current position on the grid as (x, y) coordinates.
- `world` (List[List[str]]): The grid represented as a list of lists of strings, where each string is an emoji representing the terrain.

### Returns:
- `possible_moves` (List[Tuple[int, int]]): A list of tuples representing the possible next moves.

In [8]:
def get_possible_moves(current_position: Tuple[int, int], world: List[List[str]]):
    possible_moves: List[Tuple[int,int]] = []
    for move in MOVES:
        new_x, new_y = current_position[0] + move[0], current_position[1] + move[1]
        if new_x >= 0 and new_y >= 0 and new_y < len(world) and new_x < len(world[0]):
            possible_moves.append(move)
    return possible_moves

In [9]:
#Assertion Tests
assert get_possible_moves((0, 0), small_world) == [(1, 0), (0, 1)] #Check in left corner
assert get_possible_moves((3, 3), small_world) == [(0,-1), (1,0), (0,1), (-1,0)] #Check middle spot
assert get_possible_moves((6, 6), small_world) == [(0,-1), (-1,0)] #Check right corner


<a id="manhattan_distance"></a>
## `manhattan_distance`

This function calculates the Manhattan distance between two points on a grid. The Manhattan distance is the sum of the absolute differences between the x and y coordinates of the two points.

### Parameters:
- `current` (Tuple[int, int]): The current position as (x, y) coordinates.
- `goal` (Tuple[int, int]): The goal position as (x, y) coordinates.

### Returns:
- `distance` (int): The Manhattan distance between the start and goal positions.

In [10]:
def manhattan_distance(current: Tuple[int, int], goal: Tuple[int, int]):
    return abs(current[0] - goal[0]) + abs(current[1] - goal[1])

In [11]:
# assertions/unit tests
assert manhattan_distance((0, 0), (6, 6)) == 12 # Normal distance from start for small map
assert manhattan_distance((3, 3), (0, 0)) == 6 # Half distance from middle in small map
assert manhattan_distance((6, 0), (0, 6)) == 12 # Same distance as first assert but split between two coordinates

<a id="heuristic"></a>
## `heuristic`

This function estimates the cost to reach the goal with a selected move on the grid. It considers the cost associated with moving onto different terrains (emojis), as well as the Manhattan distance to the goal. This function is used to guide the search algorithm toward the most promising paths. Also returns the move that results in the lowest estimated cost.


### Parameters:
- `current_position` (Tuple[int, int]): The current position on the grid as (x, y) coordinates.
- `goal_position` (Tuple[int, int]): The goal position on the grid as (x, y) coordinates.
- `world` (List[List[str]]): The grid represented as a list of lists of strings, where each string is an emoji representing the terrain.

### Returns:
- `move_cost` (int): The estimated cost to reach the goal with this move.

In [12]:
def heuristic(current_position: Tuple[int, int], move: Tuple[int, int], goal_position: Tuple[int, int], world: List[List[str]]):
    new_position = (current_position[0] + move[0], current_position[1] + move[1])

    # Calculate the Manhattan distance from the new position to the goal as a simple heuristic
    manhattan_distance = abs(new_position[0] - goal_position[0]) + abs(new_position[1] - goal_position[1])

    # Add any additional cost considerations based on the terrain at the new position
    terrain_cost = COSTS.get(world[new_position[1]][new_position[0]], float('inf'))

    move_cost: int = manhattan_distance + terrain_cost

    # Return the combined heuristic cost
    return move_cost

In [13]:
# assertions/unit tests
assert heuristic((0, 0), (0, 1), (6, 6), small_world) == 12 
assert heuristic((3, 3), (1, 0), (6, 6), small_world) == 6 
assert heuristic((6, 6), (0, -1), (0, 0), small_world) == 12

<a id="a_star_search"></a>
## a_star_search

The `a_star_search` function finds the optimal path from a starting point to a goal within a grid. This implementation of A* search evaluates possible paths by balancing the actual cost to reach a node and an estimated cost to reach the goal from that node. Utilizing this information 

* **world** List[List[str]]: the actual context for the navigation problem.
* **start** Tuple[int, int]: the starting location of the bot, `(x, y)`.
* **goal** Tuple[int, int]: the desired goal position for the bot, `(x, y)`.
* **costs** Dict[str, int]: is a `Dict` of costs for each type of terrain in **world**.
* **moves** List[Tuple[int, int]]: the legal movement model expressed in offsets in **world**.
* **heuristic** Callable: is a heuristic function, $h(n)$.


**returns** List[Tuple[int, int]]: the offsets needed to get from start state to the goal as a `List`.


In [14]:
def a_star_search( world: List[List[str]], start: Tuple[int, int], goal: Tuple[int, int], costs: Dict[str, int], moves: List[Tuple[int, int]], heuristic: Callable) -> List[Tuple[int, int]]:
    open_set: List[Tuple[int, Tuple[int, int]]] = [(0, start)]  # List of (f_score, position)
    came_from: Dict[Tuple[int, int], Tuple[int, int] | None] = {start: None}  # Track the path
    came_by_move: Dict[Tuple[int, int], Tuple[int, int] | None] = {}  # Track the move used to get to each position
    g_score: Dict[Tuple[int, int], float] = {start: 0}  # Cost from start to each position
    f_score: Dict[Tuple[int, int], float] = {start: heuristic(start, (0, 0), goal, world)}  # Initial heuristic cost for the start

    while open_set:
        # Sort the open set to find the position with the lowest f_score
        open_set.sort(key=lambda x: x[0])
        current_f, current_position = open_set.pop(0)  # Pop the element with the lowest f_score

        if current_position == goal:
            # Reconstruct path using moves
            move_path: List[Tuple[int, int]] = []
            while current_position != start:
                move = came_by_move[current_position]
                move_path.append(move)
                current_position = (current_position[0] - move[0], current_position[1] - move[1])
            return move_path[::-1]  # Return the move sequence in correct order

        possible_moves = get_possible_moves(current_position, world)

        for move in possible_moves:
            # Calculate the new position from this move
            new_position = (current_position[0] + move[0], current_position[1] + move[1])
            x, y = new_position
            tentative_g_score = g_score[current_position] + costs.get(world[y][x], float('inf'))

            if new_position not in g_score or tentative_g_score < g_score[new_position]:
                came_from[new_position] = current_position
                came_by_move[new_position] = move  # Track the move that led to this position
                g_score[new_position] = tentative_g_score
                f_score[new_position] = tentative_g_score + heuristic(current_position, move, goal, world)
                open_set.append((f_score[new_position], new_position))

    return None  # Return None if no path is found

In [15]:
#Assertion Tests
sample_grid = [
    ['🌾', '🌾', '🌾'],
    ['🌾', '⛰', '🌾'],
    ['🌾', '🌾', '🌾']
]

assert a_star_search(sample_grid, (0, 0), (1, 0), COSTS, MOVES, heuristic) == [(1, 0)] #Make one move to goal
assert a_star_search(sample_grid, (0, 0), (2, 2), COSTS, MOVES, heuristic) == [(1, 0), (1, 0), (0, 1), (0, 1)] #Make a full path
assert a_star_search(sample_grid, (2, 2), (10, 10), COSTS, MOVES, heuristic) == None  # Assuming out of bounds or unreachable

## pretty_print_path

The `pretty_print` function takes a 2D grid of terrain types, each represented by an emoji, and visually displays it in a structured format. It  overlays a path, generated by the `a_star_search` function, to show the route from start to goal on the grid. 

* **world** List[List[str]]: the world (terrain map) for the path to be printed upon.
* **path** List[Tuple[int, int]]: the path from start to goal, in offsets.
* **start** Tuple[int, int]: the starting location for the path.
* **goal** Tuple[int, int]: the goal location for the path.
* **costs** Dict[str, int]: the costs for each action.

**returns** int - The path cost.

In [16]:
def pretty_print_path( world: List[List[str]], path: List[Tuple[int, int]], start: Tuple[int, int], goal: Tuple[int, int], costs: Dict[str, int]) -> int:
    moves_to_emoji: Dict[Tuple[int, int], str] = {
        (1, 0): '⏩',   # Right
        (-1, 0): '⏪',  # Left
        (0, -1): '⏫',  # Up
        (0, 1): '⏬'    # Down
    }

    # Copy the world grid to modify it as to not mess up grid for other runs
    world_copy: List[List[str]] = [row[:] for row in world]

    total_cost: int = 0

    # Place the emojis along the path and calculate the total cost
    current_position: Tuple[int, int] = start
    for move in path:
        emoji: str = moves_to_emoji.get(move, '❓')  # Get the correct emoji for the move, use ? if one isnt found somehow instead of crashing
        world_copy[current_position[1]][current_position[0]] = emoji  # Place emoji at the current position

        # Calculate the next position and update the total cost
        next_position = (current_position[0] + move[0], current_position[1] + move[1])
        total_cost += costs[world[next_position[1]][next_position[0]]]

        # Update the current position
        current_position = next_position

    world_copy[goal[1]][goal[0]] = '🎁'  # Mark the goal position with a gift emoji

    # Display the modified grid
    display_emoji_grid(world_copy)

    # Return the total cost of the path
    return total_cost

In [17]:
# Assertion Tests
sample_grid = [
    ['🌾', '🌾', '🌾'],
    ['🌾', '⛰', '🌾'],
    ['🌾', '🌾', '🌾']
]

sample_start = (0, 0)
sample_goal = (len(sample_grid[0]) - 1, len(sample_grid) - 1)
sample_path = a_star_search(sample_grid, sample_start, sample_goal, COSTS, MOVES, heuristic)

assert pretty_print_path(sample_grid, [], sample_start, sample_goal, COSTS) == 0 # Check goal emoji added
assert pretty_print_path(sample_grid, [(0,1)], sample_start, sample_goal, COSTS) == 1 # One move down
assert pretty_print_path(sample_grid, sample_path, sample_start, sample_goal, COSTS) == 4 # Full Path printing

🌾,🌾,🌾
🌾,⛰,🌾
🌾,🌾,🎁


⏬,🌾,🌾
🌾,⛰,🌾
🌾,🌾,🎁


⏩,⏩,⏬
🌾,⛰,⏬
🌾,🌾,🎁



<div style="background: lemonchiffon; margin:20px; padding: 20px;">
    <strong>Important</strong>
    <p>
        Does your output of pretty_print_path really look like the specification? Go check again.
    </p>
</div>

## Problems

## Problem 1. 

Execute `a_star_search` and `pretty_print_path` for the `small_world`.

If you change any values while developing your code, make sure you change them back! (Better yet, don't do it. Copy them elsewhere and change the values, then delete those experiments).

In [18]:
small_start = (0, 0)
small_goal = (len(small_world[0]) - 1, len(small_world) - 1)
small_path = a_star_search(small_world, small_start, small_goal, COSTS, MOVES, heuristic)
small_path_cost = pretty_print_path(small_world, small_path, small_start, small_goal, COSTS)
print(f"total path cost: {small_path_cost}")
print(small_path)

⏬,🌲,🌲,🌲,🌲,🌲,🌲
⏬,🌲,🌲,🌲,🌲,🌲,🌲
⏬,🌲,🌲,🌲,🌲,🌲,🌲
⏩,⏩,⏩,⏩,⏩,⏩,⏬
🌲,🌲,🌲,🌲,🌲,🌲,⏬
🌲,🌲,🌲,🌲,🌲,🌲,⏬
🌲,🌲,🌲,🌲,🌲,🌲,🎁


total path cost: 12
[(0, 1), (0, 1), (0, 1), (1, 0), (1, 0), (1, 0), (1, 0), (1, 0), (1, 0), (0, 1), (0, 1), (0, 1)]


## Problem 2

Execute `a_star_search` and `print_path` for the `full_world`

In [19]:
full_start = (0, 0)
full_goal = (len(full_world[0]) - 1, len(full_world) - 1)
full_path = a_star_search(full_world, full_start, full_goal, COSTS, MOVES, heuristic)
full_path_cost = pretty_print_path(full_world, full_path, full_start, full_goal, COSTS)
print(f"total path cost: {full_path_cost}")
print(full_path)

⏩,⏩,⏩,⏩,⏩,⏩,⏩,⏩,⏩,⏩,⏩,⏩,⏩,⏩,⏩,⏩,⏩,⏩,⏩,⏩,⏩,⏩,⏬,🌾,⛰,⛰,⛰
🌾,🌾,🌾,🌾,🌾,⛰,⛰,🌾,🌾,🌾,🌾,🐊,🐊,🐊,🌾,🌾,🌾,🌾,🌾,🌾,🌾,🌾,⏬,⛰,🌋,🌋,⛰
🌾,🌾,🌾,🌾,🌾,⛰,⛰,⛰,🌾,🌾,🐊,🐊,🐊,🐊,🌾,🌾,🌋,🌾,🌾,🌾,⛰,🌾,⏬,⛰,⛰,🌋,🌾
🌾,🌾,🌾,🌾,⛰,⛰,🌋,⛰,⛰,🐊,🐊,🐊,🐊,🐊,🌾,🌋,🌋,🌋,🌋,🌾,⛰,🌾,⏩,⏬,⛰,🌋,🌾
🌾,🌾,🌋,⛰,⛰,🌋,🌋,⛰,⛰,🐊,🐊,🐊,🐊,🐊,🌋,🌋,🌲,🌋,🌋,🌋,⛰,⛰,🌾,⏩,⏩,⏩,⏬
🌲,🌾,🌋,🌋,🌋,🌋,⛰,⛰,⛰,🐊,🐊,🐊,🌾,🌾,🌾,🌋,🌲,🌲,🌋,🌋,⛰,⛰,🌾,⛰,⛰,🌾,⏬
🌲,🌾,🌲,🌋,🌋,⛰,⛰,⛰,🌾,🌾,🐊,🌾,🌾,🌾,🌾,🌲,🌲,🌲,🌲,🌋,🌋,⛰,⛰,⛰,🌾,🌾,⏬
🌲,🌲,🌲,🌋,🌲,⛰,🌾,🌾,🌾,🌾,🌾,🌾,⛰,⛰,🌲,🌲,🌲,🌲,🌲,🌲,🌋,🌋,⛰,⛰,🌾,🌾,⏬
🌲,🌲,🌲,🌲,🌲,🌾,🌾,🌾,🌾,⛰,⛰,⛰,⛰,🌲,🌲,🌲,🌲,🌲,🌲,🌲,🌲,🌋,⛰,⛰,🌾,🌾,⏬
🌲,🌲,🌲,🌲,🌾,🌾,🌾,🌾,🌾,⛰,⛰,🌋,🌋,🌲,🌲,🌲,🌲,🌲,🌲,🌲,🌲,🌋,🌋,⛰,🌾,🌾,⏬
🌲,🌲,🌲,🌲,🌾,🌾,🌾,🌾,🌾,⛰,🌋,🌋,🌋,⛰,🌲,🌲,🌲,🌲,🌲,🌲,🌲,🌲,🌋,🌾,🌾,⛰,⏬


total path cost: 98
[(1, 0), (1, 0), (1, 0), (1, 0), (1, 0), (1, 0), (1, 0), (1, 0), (1, 0), (1, 0), (1, 0), (1, 0), (1, 0), (1, 0), (1, 0), (1, 0), (1, 0), (1, 0), (1, 0), (1, 0), (1, 0), (1, 0), (0, 1), (0, 1), (0, 1), (1, 0), (0, 1), (1, 0), (1, 0), (1, 0), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1)]


## Comments

(This is the place to leave me comments)

## To think about for future assignments...

This first assignment may not have been difficult for you if you've encountered A* search before in your Algorithms course. In preparation for future assignments that build on State Space Search, you can think about the following or even do an implementation if you like. You should **not** submit it as part of this assignment.

In several future assignments, we will have a need for a "plain ol'" Depth First Search algorithm.

1. Implement DFS Search to solve the problem presented in this programming assignment. Try to be as general as possible (don't hard code anything you can pass as a formal parameter).
2. Can you implement DFS Search as a higher order function and supply your own `is_goal`, `successors`, and `path` functions? How do you handle *state*?
3. Can you write a version of DFS that returns all the solutions?

In one future assignment a Breadth First Search algorithm will be very handy. Can you implement a search algorithm that changes whether it uses DFS or BFS by parameterization?

## Before You Submit...

1. Did you provide output exactly as requested?
2. Did you follow the Programming Requirements on Canvas?

Do not submit any other files.